# ecallisto-fits Plotting Tutorial

This notebook demonstrates all the plotting capabilities of the ecallisto-fits library, including:

- **Raw spectrum plotting** - View unprocessed data
- **Customizable plot parameters** - Control colormaps, clipping values, figure size, and more
- **Background subtraction** - Visualize data before clipping
- **Time axis conversion** - Switch between seconds and UT format

## Setup

In [ ]:
import ecallisto_fits as ecf
import matplotlib.pyplot as plt
import numpy as np

# For inline plots
%matplotlib inline

## 1. Load Sample Data

First, let's load a sample FITS file. Make sure you have a sample file in the `data/` directory.

In [ ]:
# Load the sample FITS file
ds = ecf.read_fits("data/SAMPLE_20240101_120000_01.fit.gz")

print(f"Data shape: {ds.shape}")
print(f"Frequency range: {ds.freqs_mhz.min():.1f} - {ds.freqs_mhz.max():.1f} MHz")
print(f"Time range: {ds.time_s.min():.1f} - {ds.time_s.max():.1f} seconds")
print(f"Metadata: {ds.meta}")

## 2. Plot Raw Spectrum

Use `plot_raw_spectrum()` to visualize the data without any processing.

In [ ]:
# Basic raw spectrum plot
fig, ax, im = ecf.plot_raw_spectrum(ds, title="Raw Spectrum")
plt.show()

In [ ]:
# Customized raw spectrum with figure size and colormap
fig, ax, im = ecf.plot_raw_spectrum(
    ds,
    title="Raw Spectrum (Custom)",
    cmap="plasma",
    figsize=(14, 6),
)
plt.show()

## 3. Background Subtraction (Before Clipping)

The `background_subtract()` function removes the mean per frequency channel without applying clipping. This helps visualize deviations from the baseline.

In [ ]:
# Apply background subtraction only (no clipping)
ds_bg = ecf.background_subtract(ds)

# Verify each frequency channel now has zero mean
print(f"Mean per channel (should be ~0): {ds_bg.data.mean(axis=1)[:5]}")
print(f"Processing metadata: {ds_bg.meta.get('processing')}")

In [ ]:
# Use the convenience function to plot background subtracted data
fig, ax, im = ecf.plot_background_subtracted(
    ds,
    title="Background Subtracted (Before Clipping)",
    vmin=-20,
    vmax=40,
    cmap="RdBu_r",  # Diverging colormap shows +/- deviations
    figsize=(14, 6),
)
plt.show()

## 4. Noise Reduction with Custom Clipping

Apply the full noise reduction pipeline (background subtraction + clipping + scaling).

In [ ]:
# Apply noise reduction with custom clipping values
ds_reduced = ecf.noise_reduce_mean_clip(
    ds,
    clip_low=-5.0,
    clip_high=20.0,
    scale=None  # Disable scaling for this example
)

print(f"Processing metadata: {ds_reduced.meta.get('noise_reduction')}")

## 5. Customizable Plot Parameters

`plot_dynamic_spectrum()` now supports:
- `vmin` / `vmax` - Colormap clipping values
- `figsize` - Figure size in inches
- `cmap` - Any matplotlib colormap
- `**imshow_kwargs` - Any additional matplotlib imshow parameters

In [ ]:
# Plot with custom clipping values and colormap
fig, ax, im = ecf.plot_dynamic_spectrum(
    ds_reduced,
    title="Noise Reduced (vmin=-5, vmax=20)",
    vmin=-5,
    vmax=20,
    cmap="inferno",
    figsize=(14, 6),
)
plt.show()

In [ ]:
# Try different colormaps and clipping values
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

configs = [
    {"title": "Inferno (default)", "cmap": "inferno", "vmin": -5, "vmax": 20},
    {"title": "Viridis", "cmap": "viridis", "vmin": -5, "vmax": 20},
    {"title": "Magma (tight clip)", "cmap": "magma", "vmin": 0, "vmax": 10},
    {"title": "Plasma (wide clip)", "cmap": "plasma", "vmin": -10, "vmax": 30},
]

for ax, cfg in zip(axes.flat, configs):
    ecf.plot_dynamic_spectrum(
        ds_reduced,
        ax=ax,
        title=cfg["title"],
        cmap=cfg["cmap"],
        vmin=cfg["vmin"],
        vmax=cfg["vmax"],
    )

plt.tight_layout()
plt.show()

In [ ]:
# Pass additional matplotlib imshow parameters
fig, ax, im = ecf.plot_dynamic_spectrum(
    ds_reduced,
    title="With Bilinear Interpolation",
    vmin=-5,
    vmax=20,
    cmap="inferno",
    figsize=(14, 6),
    interpolation="bilinear",  # Smooth interpolation
    alpha=0.9,  # Slight transparency
)
plt.show()

## 6. Time Axis Formats

Display time in seconds or UT (Universal Time) format using the `time_format` parameter.

In [ ]:
# Compare time formats side by side
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# Time in seconds (default)
ecf.plot_dynamic_spectrum(
    ds_reduced,
    ax=axes[0],
    title="Time in Seconds",
    time_format="seconds",
    vmin=-5,
    vmax=20,
)

# Time in UT format
ecf.plot_dynamic_spectrum(
    ds_reduced,
    ax=axes[1],
    title="Time in UT",
    time_format="ut",
    vmin=-5,
    vmax=20,
)

plt.tight_layout()
plt.show()

## 7. TimeAxisConverter

The `TimeAxisConverter` class provides programmatic conversion between elapsed seconds and UT time.

In [ ]:
# Create a converter from the spectrum's metadata
converter = ecf.TimeAxisConverter.from_dynamic_spectrum(ds)

print(f"UT start time: {converter.ut_start_sec} seconds since midnight")
print(f"UT start time: {converter.seconds_to_ut(0)}")

In [ ]:
# Convert seconds to UT
test_times = [0, 60, 100, 300, 600]

print("Seconds → UT conversion:")
for t in test_times:
    ut = converter.seconds_to_ut(t)
    print(f"  {t:5d} seconds → {ut}")

In [ ]:
# Convert UT back to seconds
test_ut_times = ["12:00:00", "12:01:00", "12:05:00", "12:10:00"]

print("UT → Seconds conversion:")
for ut in test_ut_times:
    seconds = converter.ut_to_seconds(ut)
    print(f"  {ut} → {seconds:.1f} seconds")

In [ ]:
# Create a converter manually
manual_converter = ecf.TimeAxisConverter(ut_start_sec=43200.0)  # 12:00:00

print(f"Manual converter start: {manual_converter.seconds_to_ut(0)}")
print(f"After 1 hour: {manual_converter.seconds_to_ut(3600)}")

## 8. Complete Workflow

Putting it all together - a complete analysis workflow.

In [ ]:
# Load data
ds = ecf.read_fits("data/SAMPLE_20240101_120000_01.fit.gz")
print(f"Loaded: {ds.meta.get('station')} observation from {ds.meta.get('date')}")

# Create figure with 4 panels
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Raw data
ecf.plot_raw_spectrum(
    ds,
    ax=axes[0, 0],
    title="1. Raw Spectrum",
    cmap="viridis",
)

# 2. Background subtracted (before clipping)
ecf.plot_background_subtracted(
    ds,
    ax=axes[0, 1],
    title="2. Background Subtracted",
    vmin=-20,
    vmax=40,
    cmap="RdBu_r",
)

# 3. Noise reduced (default params)
ds_reduced = ecf.noise_reduce_mean_clip(ds)
ecf.plot_dynamic_spectrum(
    ds_reduced,
    ax=axes[1, 0],
    title="3. Noise Reduced (Default)",
    cmap="inferno",
)

# 4. Noise reduced with custom clipping + UT time
ds_custom = ecf.noise_reduce_mean_clip(ds, clip_low=-2, clip_high=15, scale=None)
ecf.plot_dynamic_spectrum(
    ds_custom,
    ax=axes[1, 1],
    title="4. Custom Clipping + UT Time",
    vmin=-2,
    vmax=15,
    cmap="magma",
    time_format="ut",
)

plt.tight_layout()
plt.savefig("plotting_demo.png", dpi=150, bbox_inches="tight")
plt.show()

print("\nSaved figure to 'plotting_demo.png'")

## Summary

This notebook demonstrated:

| Feature | Function | Description |
|---------|----------|-------------|
| Raw plotting | `plot_raw_spectrum()` | Visualize unprocessed data |
| Background subtraction | `background_subtract()` | Mean removal without clipping |
| Background plot | `plot_background_subtracted()` | Convenience function |
| Custom clipping | `vmin`, `vmax` params | Control colormap range |
| Figure size | `figsize` param | Set plot dimensions |
| Custom colormap | `cmap` param | Any matplotlib colormap |
| UT time format | `time_format="ut"` | Display time in HH:MM:SS |
| Time conversion | `TimeAxisConverter` | Seconds ↔ UT programmatically |
| Extra params | `**imshow_kwargs` | interpolation, alpha, etc. |